This Notebook will be used for the capstone project

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import folium

In [2]:
import geopy
from geopy import geocoders
from geopy.geocoders import Nominatim

### Pulling data from FourSquare API and putting it into a Pandas DataFrame

In [3]:
client_id = '[FourSquare API ID here]'
client_secret = '[Foursquare Client Secret here]'
api_v = '20190425'
intent = 'browse'
category = '4bf58dd8d48988d1e0931735,5e18993feee47d000759b256,5665c7b9498e7d8a4f2c0f06' #Category IDs of Coffee Shops
location = 'Charlotte,NC'
search_radius = 80000 # in meters
limit = 500

In [4]:
fs_call_str = 'https://api.foursquare.com/v2/venues/search?client_id={client_id}&client_secret={client_secret}&v={v}&near={loc}&intent={intent}&radius={rad}&limit={lim}&categoryId={cat}'.format(client_id = client_id, client_secret = client_secret, v = api_v, loc = location, intent = intent, rad = search_radius, lim = limit, cat = category)

response = requests.get(fs_call_str)
json_response = response.json()

In [5]:
venues_dict = json_response['response']['venues']

venue_ids = list()
venue_names = list()
venue_postal_codes = list()

for venue in venues_dict:
    if 'postalCode' in venue['location'].keys():
        venue_postal_codes.append(venue['location']['postalCode'])
        venue_ids.append(venue['id'])
        venue_names.append(venue['name'])
        
dict_for_pandas = {'Venue_ID': venue_ids, 'Name': venue_names, 'Postal_Code': venue_postal_codes}
df = pd.DataFrame(data = dict_for_pandas)

print('There are {ct} coffee shops in Charlotte.'.format(ct = df.shape[0]))

There are 48 coffee shops in Charlotte.


### Pulling the Census data for the revlevant postal codes

In [6]:
cb_api_key = '[Census Bureau API key here]'

In [7]:
def census_api_call(zip_code, census_api_key):
    '''Returns the population of the zip code in question as an integer'''
    zip_code = str(zip_code)
    cb_call_str = 'https://api.census.gov/data/2017/acs/acs5?get=NAME,B01003_001E&for=zip%20code%20tabulation%20area:{zcta}&key={key}'.format(zcta = zip_code, key = census_api_key)
    cb_response = requests.get(cb_call_str)
    cb_response_list = json.loads(cb_response.text)
    
    return int(cb_response_list[1][1])

In [8]:
zips = df['Postal_Code'].unique().tolist()
print(len(zips))
pops = list()

for zip_code in zips:
    pops.append(census_api_call(zip_code, cb_api_key))

df_pops = pd.DataFrame({'Postal_Code': zips, 'Population': pops})

28


### Merging the Datasets

In [9]:
df_shops = df.groupby(['Postal_Code'], as_index = False)['Venue_ID'].count()
df_shops.rename(columns = {'Venue_ID': 'Venue_Ct'}, inplace = True)

df_merged = pd.merge(left = df_shops, right = df_pops, on = 'Postal_Code', how = 'inner')
df_merged['Pop_Per_Venue'] = df_merged['Population'].div(df_merged['Venue_Ct'])

### Put merged DataFrame onto a map in Folium

In [10]:
nc_zip_geojson = 'nc_north_carolina_zip_codes_geo.min.json'

with open(nc_zip_geojson) as f:
    nc_zip_boundaries = json.load(f)

    
# The nc_zip_geojson gives info for all zip codes in North Carolina. For the resulting map, we only want zip codes where there are coffee shops, so we need to cut it down
clt_dict = dict()
clt_dict['type'] = nc_zip_boundaries['type']

zip_props_list = list()

for zip_prop in nc_zip_boundaries['features']:
    cur_zip = zip_prop['properties']['ZCTA5CE10']
    if cur_zip in zips:
        zip_props_list.append(zip_prop)

clt_dict['features'] = zip_props_list

In [11]:
user_agent_input = '[your email address here]'

geolocator = Nominatim(user_agent = user_agent_input)
loc = geolocator.geocode(location)
lat = float(loc.raw['lat']) 
lon = float(loc.raw['lon'])

m = folium.Map(location = [lat, lon], zoom_start = 9)

folium.Choropleth(
    geo_data = clt_dict,
    data = df_merged,
    columns=['Postal_Code', 'Pop_Per_Venue'],
    key_on = 'feature.properties.ZCTA5CE10',
    fill_color='RdPu',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name = 'People per Coffee Shop',
    reset=True
).add_to(m)

display(m)
print("If you're opening a coffee shop, you would want to target an area that has a lot of people per existing coffee shop (or no coffee shop). The dark-colored areas and non-colored areas are currently under-served, so there may be opportunity there.")

If you're opening a coffee shop, you would want to target an area that has a lot of people per existing coffee shop (or no coffee shop). The dark-colored areas and non-colored areas are currently under-served, so there may be opportunity there.
